In [ ]:
import numpy as np                              #数値計算ライブラリ
from math import sin, cos, pi                   #低機能だが計算が速い数学関数
from scipy.integrate import odeint              #常微分方程式ライブラリ
import matplotlib.pyplot as plt                 #描画ライブラリ
from mpl_toolkits.mplot3d import Axes3D         #3次元座標用のモジュール
from mpl_toolkits.mplot3d.art3d import Poly3DCollection #3次元ポリゴン用のモジュール
from matplotlib.animation import FuncAnimation  #アニメーションライブラリ
from matplotlib import rc                       #グラフ調整ライブラリ

### 運動学用の関数

In [ ]:
E3 = np.eye(3)
E4 = np.eye(4)

def cross_mat(vec):
    '''
    外積行列(ベクトル)
    '''
    return np.array([
        [ 0, -vec[2],  vec[1]],
        [ vec[2],  0, -vec[0]],
        [-vec[1],  vec[0],  0],        
    ])

def Rot_from_quat(q):
    '''
    回転行列をクオータニオンから作る
    '''
    qvec = q[1:]
    return np.array(
        (2*q[0]**2 - 1)*E3 + 2*np.outer(qvec,qvec) + 2*q[0]*cross_mat(qvec)
    )

def quaternion_from_axis_angle(axis, angle):
    '''
    クォータニオンを(回転軸，回転角)から作る
    '''
    half = 0.5*angle
    cos_half = cos(half)
    sin_half = sin(half)
    return np.array([
        cos_half,
        axis[0]*sin_half,
        axis[1]*sin_half,
        axis[2]*sin_half,
    ])

def H_from_Rot_trans(Rot, vec):
    '''
    同次変換行列を(回転行列，変位ベクトル)から作る
    '''
    H = np.hstack((Rot,np.array([vec]).T))
    H = np.vstack((H,[0,0,0,1]))
    
    return H



### ポリゴンデータの操作

In [ ]:
def poly_multiply_H(poly_list, H):
    '''
    同次変換行列 H をポリゴンのリスト poly_list に掛ける
    '''
    POLY_LIST = []
    for poly in poly_list: #ポリゴン１枚の頂点セット in そのリスト
        
        POLY = []
        for point in poly: #各頂点 in あるポリゴンの頂点セット
            Hpoint = np.hstack((point,[1])) #同次座標化
            Hpoint = np.dot(H,Hpoint)       #同次変換
            point = Hpoint[:-1]             #3次元座標化
            POLY.append(point)
            
        POLY_LIST.append(POLY)
        
    return POLY_LIST

def poly_Trans(poly_list, r):
    '''
    同次変換行列 H をポリゴンのリスト poly_list に掛ける
    '''
    POLY_LIST = []
    for poly in poly_list: #ポリゴン１枚の頂点セット in そのリスト
        
        POLY = []
        for point in poly: #各頂点 in あるポリゴンの頂点セット
            POLY.append(point + r)
            
        POLY_LIST.append(POLY)
        
    return POLY_LIST

def poly_plot(ax, poly):
    facecolors=['C1','C2','C6','C6','C3']
    for i, p in enumerate(poly):
        pc = Poly3DCollection([p])
        pc.set_alpha(0.7) #alpha first
        pc.set_facecolor(facecolors[i])
        ax.add_collection3d(pc)

def poly_plot_H(ax, poly, H):
    poly = poly_multiply_H(poly, H)
    poly_plot(ax, poly)
        
def poly_plot_quat_trans(ax, poly, q, vec):
    Rot = Rot_from_quat(q)
    H = H_from_Rot_trans(Rot,vec)
    poly_plot_H(ax, poly, H)

### ポリゴンのデータサンプル

In [ ]:
def poly_triangle(w, d, h):
    '''
    厚みのある二等辺三角形 (幅, 厚, 高)
    '''
    A = np.array([ w/2, -d/2, 0])
    B = np.array([ w/2,  d/2, 0])
    C = np.array([-w/2,  d/2, 0])
    D = np.array([-w/2, -d/2, 0])
    E = np.array([   0, -d/2, h])
    F = np.array([   0,  d/2, h])
    
    verts =[
        [F, B, C],    #前面
        [E, D, A],    #後面
        [A, B, F, E], #斜面
        [C, D, E, F], #斜面
        [A, B, C, D], #底面
    ]
    
    return verts

### お試しプロット

In [ ]:
# 基準姿勢データのサンプル
body00 = poly_triangle(w=1, d=0.1, h=0.5)
body0 = poly_Trans(body00, [0, 0, -0.5/3])

# 同次行列 H のサンプル
q = quaternion_from_axis_angle( [1,0,0], pi/2 )
r = [0,0,0]
Rot = Rot_from_quat(q)
H = H_from_Rot_trans(Rot,r)

# 基準姿勢データを同次変換
body = poly_multiply_H(body0, H)

# print(body0)
# print(body)

%matplotlib inline
fig = plt.figure(figsize=plt.figaspect(1))
ax = fig.add_subplot (1, 1, 1, projection = '3d')

poly_plot_H(ax, body0, E4)
poly_plot_quat_trans(ax, body, q, r)

ax.set_xlim(-1,1)
ax.set_ylim(-1,1)
ax.set_zlim(-1,1)
ax.set_xlabel('X')
ax.set_ylabel('Y')
plt.show()

## 主軸オイラー方程式

In [ ]:
def EulerEQ(om, diagI):
    return -np.array([
      (diagI[2]-diagI[1])*om[1]*om[2]/diagI[0],  
      (diagI[0]-diagI[2])*om[2]*om[0]/diagI[1],  
      (diagI[1]-diagI[0])*om[0]*om[1]/diagI[2],  
    ])

def Kq(q):
    return 0.5*np.array([
        [-q[1], -q[2], -q[3]],
        [ q[0], -q[3],  q[2]],
        [ q[3],  q[0], -q[1]],
        [-q[2],  q[1],  q[0]],
    ])

def EOM(x, t, diagI):
    om = x[0:3]
    q  = x[3:]
    dom = EulerEQ(om, diagI)
    dq = np.dot(Kq(q), om)

    return np.hstack((dom,dq))

### 動画の条件

In [ ]:
tennis_diagI = np.array([
    12, 2, 10
])

tennis_x0 = np.array([
    1, 1, -7,
    1, 0, 0, 0,
])

tennis_dt = 0.02

### 時間応答

In [ ]:
dt = tennis_dt
x0 = tennis_x0
diagI = tennis_diagI

tN = 1200
ts = np.linspace(0, (tN-1)*dt, tN)

sol = odeint(EOM, x0, ts, args=(diagI,))

In [ ]:
fig = plt.figure(figsize=(5,2.5*7))
axs = fig.subplots(nrows=7)

for i, ax in enumerate(axs):
    ax.plot(ts, sol[:,i])
    ax.set_xlabel('t')
    if i<3:
        ax.set_ylabel('$\omega_%d$'%(i+1))
    else:
        ax.set_ylabel('$q_%d$'%(i-3))

In [ ]:
quats = sol[::3,3:] #アニメーションはデータ3個とばし

In [ ]:
fig = plt.figure(figsize=plt.figaspect(1))
ax = fig.add_subplot(1, 1, 1, projection = '3d')

def each_frame(idx):
    ax.cla()
    poly_plot_quat_trans(ax, body0, quats[idx], [0,0,0])
    ax.set_xlim(-1,1)
    ax.set_ylim(-1,1)
    ax.set_zlim(-1,1)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')

anim = FuncAnimation(
    fig, each_frame, 
    interval=80, frames=len(quats)
)

rc('animation', html='jshtml')
anim